In [12]:
### Imports and data ###

#Imports
import tensorflow as tf
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import SGD
import matplotlib.pyplot as plt
import numpy as np

import datetime as dt
import matplotlib as style
import pandas_datareader.data as web

#Get Data from yahoo

start = dt.datetime(2000,1,1)
end = dt.datetime(2016,12,31)
shift = 100
df = web.DataReader('AAPL', 'yahoo', start, end)[['High','Low','Open','Close']]
df_target = df[['Close']].shift(-shift)

#Get the input and output data and scale for 0-1
x_data = df.values[:-shift]
y_data = df_target.values[:-shift]
max_val = max(x_data.max(), y_data.max())

x_data /= max_val
y_data /= max_val

#Split the data for training and testing
data_split = 0.75

x_train = x_data[:int(len(x_data)*data_split)]
x_test = x_data[int(len(x_data)*data_split):]
y_train = y_data[:int(len(y_data)*data_split)]
y_test = y_data[int(len(y_data)*data_split):]

#Generate batches of inputs and outputs
def batch_generator(batch_size, sequence_length):
    while True:
        x_batch = np.zeros(shape=(batch_size, sequence_length, x_data.shape[1]), dtype=np.float16)
        y_batch = np.zeros(shape=(batch_size, sequence_length, y_data.shape[1]), dtype=np.float16)
        
        for i in range(batch_size):
            idx = np.random.randint(len(x_train)-sequence_length)
            
            x_batch[i] = x_train[idx:idx+sequence_length]
            y_batch[i] = y_train[idx:idx+sequence_length]
            
        yield(x_batch, y_batch)

#Get sample batch

generator = batch_generator(320,100)
x_temp = np.zeros(shape=(x_test.shape[0]-100, 100, x_test.shape[1]), dtype=np.float16)
y_temp = np.zeros(shape=(y_test.shape[0]-100, 100, y_test.shape[1]), dtype=np.float16)

for i in range(len(x_test)-100):
    x_temp[i] = x_test[i:i+100]
    y_temp[i] = y_test[i:i+100]
val_data = (x_temp, np.expand_dims(y_temp, axis=0))

AttributeError: module 'numpy' has no attribute 'Flatten'

In [15]:
#Build the model

model = Sequential()

model.add(LSTM(128, input_shape=(100,4), activation='relu', return_sequences=True))
model.add(Dropout(.02))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(.02))

model.add(Dense(32, activation='relu'))
model.add(Dropout(.02))

model.add(Dense(100, activation='relu'))

opt=SGD(lr=.01, decay=1e-6)
model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])

In [16]:
#Fit and test the model

print(next(generator)[0].shape)
print(val_data[1].shape)
print(y_test.shape)

model.fit_generator(generator=generator, epochs=20, steps_per_epoch=100, validation_data=val_data)

(320, 100, 4)
(1, 945, 100, 1)
(1045, 1)


ValueError: Error when checking target: expected dense_10 to have 2 dimensions, but got array with shape (1, 945, 100, 1)